# Getting started with Word2Vec in Gensim and making it work!

The idea behind Word2Vec is pretty simple. We are making and assumption that you can tell the meaning of a word by the company it keeps. This is analogous to the saying *show me your friends, and I'll tell who you are*. So if you have two words that have very similar neighbors (i.e. the usage context is about the same), then these words are probably quite similar in meaning or are at least highly related. For example, the words `shocked`,`appalled` and `astonished` are typically used in a similar context. 

In this tutorial, you will learn how to use the Gensim implementation of Word2Vec and actually get it to work! I have heard a lot of complaints about poor performance etc, but its really a combination of two things, (1) your input data and (2) your parameter settings. Note that the training algorithms in this package were ported from the [original Word2Vec implementation by Google](https://arxiv.org/pdf/1301.3781.pdf) and extended with additional functionality.

### Imports and logging

First, we start with our imports and get logging established:

In [1]:
# imports needed and set up logging
import gzip
import gensim 
import logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)


### Dataset 
Next, is our dataset. The secret to getting Word2Vec really working for you is to have lots and lots of text data. In this case I am going to use data from the [OpinRank](http://kavita-ganesan.com/entity-ranking-data/) dataset. This dataset has full user reviews of cars and hotels. I have specifically concatenated all of the hotel reviews into one big file which is about 97MB compressed and 229MB uncompressed. We will use the compressed file for this tutorial. Each line in this file represents a hotel review. You can download the OpinRank Word2Vec dataset here.

To avoid confusion, while gensim’s word2vec tutorial says that you need to pass it a sequence of sentences as its input, you can always pass it a whole review as a sentence (i.e. a much larger size of text), and it should not make much of a difference. 

Now, let's take a closer look at this data below by printing the first line. You can see that this is a pretty hefty review.

In [2]:
data_file="reviews_data.txt.gz"

with gzip.open ('reviews_data.txt.gz', 'rb') as f:
    for i,line in enumerate (f):
        print(line)
        break


b"Oct 12 2009 \tNice trendy hotel location not too bad.\tI stayed in this hotel for one night. As this is a fairly new place some of the taxi drivers did not know where it was and/or did not want to drive there. Once I have eventually arrived at the hotel, I was very pleasantly surprised with the decor of the lobby/ground floor area. It was very stylish and modern. I found the reception's staff geeting me with 'Aloha' a bit out of place, but I guess they are briefed to say that to keep up the coroporate image.As I have a Starwood Preferred Guest member, I was given a small gift upon-check in. It was only a couple of fridge magnets in a gift box, but nevertheless a nice gesture.My room was nice and roomy, there are tea and coffee facilities in each room and you get two complimentary bottles of water plus some toiletries by 'bliss'.The location is not great. It is at the last metro stop and you then need to take a taxi, but if you are not planning on going to see the historic sites in Be

### Read files into a list
Now that we've had a sneak peak of our dataset, we can read it into a list so that we can pass this on to the Word2Vec model. Notice in the code below, that I am directly reading the 
compressed file. I'm also doing a mild pre-processing of the reviews using `gensim.utils.simple_preprocess (line)`. This does some basic pre-processing such as tokenization, lowercasing, etc and returns back a list of tokens (words). Documentation of this pre-processing method can be found on the official [Gensim documentation site](https://radimrehurek.com/gensim/utils.html). 



In [3]:

def read_input(input_file):
    """This method reads the input file which is in gzip format"""
    
    logging.info("reading file {0}...this may take a while".format(input_file))
    
    with gzip.open (input_file, 'rb') as f:
        for i, line in enumerate (f): 

            if (i%10000==0):
                logging.info ("read {0} reviews".format (i))
            # do some pre-processing and return a list of words for each review text
            yield gensim.utils.simple_preprocess (line)

# read the tokenized reviews into a list
# each review item becomes a serries of words
# so this becomes a list of lists
documents = list (read_input (data_file))
logging.info ("Done reading data file")    

2018-01-28 00:27:46,482 : INFO : reading file reviews_data.txt.gz...this may take a while
2018-01-28 00:27:46,484 : INFO : read 0 reviews
2018-01-28 00:27:48,868 : INFO : read 10000 reviews
2018-01-28 00:27:51,350 : INFO : read 20000 reviews
2018-01-28 00:27:54,287 : INFO : read 30000 reviews
2018-01-28 00:27:57,177 : INFO : read 40000 reviews
2018-01-28 00:28:00,147 : INFO : read 50000 reviews
2018-01-28 00:28:03,028 : INFO : read 60000 reviews
2018-01-28 00:28:05,508 : INFO : read 70000 reviews
2018-01-28 00:28:08,176 : INFO : read 80000 reviews
2018-01-28 00:28:10,532 : INFO : read 90000 reviews
2018-01-28 00:28:12,768 : INFO : read 100000 reviews
2018-01-28 00:28:14,962 : INFO : read 110000 reviews
2018-01-28 00:28:17,314 : INFO : read 120000 reviews
2018-01-28 00:28:19,624 : INFO : read 130000 reviews
2018-01-28 00:28:21,985 : INFO : read 140000 reviews
2018-01-28 00:28:24,178 : INFO : read 150000 reviews
2018-01-28 00:28:26,464 : INFO : read 160000 reviews
2018-01-28 00:28:29,481

## Training the Word2Vec model

Training the model is fairly straightforward. You just instantiate Word2Vec and pass the reviews that we read in the previous step (the `documents`). So, we are essentially passing on a list of lists. Where each list within the main list contains a set of tokens from a user review. Word2Vec uses all these tokens to internally create a vocabulary. And by vocabulary, I mean a set of unique words.

After building the vocabulary, we just need to call `train(...)` to start training the Word2Vec model. Training on the [OpinRank](http://kavita-ganesan.com/entity-ranking-data/) dataset takes about 10 minutes so please be patient while running your code on this dataset.

Behind the scenes we are actually training a simple neural network with a single hidden layer. But, we are actually not going to use the neural network after training. Instead, the goal is to learn the weights of the hidden layer. These weights are essentially the word vectors that we’re trying to learn. 

In [11]:
model = gensim.models.Word2Vec (documents, size=150, window=10, min_count=2, workers=10)
model.train(documents,total_examples=len(documents),epochs=10)

2018-01-26 22:57:00,707 : WARNING : consider setting layer size to a multiple of 4 for greater performance
2018-01-26 22:57:00,709 : INFO : collecting all words and their counts
2018-01-26 22:57:00,710 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-01-26 22:57:01,045 : INFO : PROGRESS: at sentence #10000, processed 1655714 words, keeping 25777 word types
2018-01-26 22:57:01,399 : INFO : PROGRESS: at sentence #20000, processed 3317863 words, keeping 35016 word types
2018-01-26 22:57:01,803 : INFO : PROGRESS: at sentence #30000, processed 5264072 words, keeping 47518 word types
2018-01-26 22:57:02,188 : INFO : PROGRESS: at sentence #40000, processed 7081746 words, keeping 56675 word types
2018-01-26 22:57:02,560 : INFO : PROGRESS: at sentence #50000, processed 9089491 words, keeping 63744 word types
2018-01-26 22:57:02,905 : INFO : PROGRESS: at sentence #60000, processed 11013723 words, keeping 76781 word types
2018-01-26 22:57:03,202 : INFO : PROGRESS: a

2018-01-26 22:57:51,698 : INFO : PROGRESS: at 35.68% examples, 1349834 words/s, in_qsize 17, out_qsize 2
2018-01-26 22:57:52,698 : INFO : PROGRESS: at 36.50% examples, 1346949 words/s, in_qsize 18, out_qsize 1
2018-01-26 22:57:53,708 : INFO : PROGRESS: at 37.32% examples, 1344214 words/s, in_qsize 19, out_qsize 0
2018-01-26 22:57:54,718 : INFO : PROGRESS: at 38.11% examples, 1338556 words/s, in_qsize 19, out_qsize 0
2018-01-26 22:57:55,722 : INFO : PROGRESS: at 38.98% examples, 1336874 words/s, in_qsize 16, out_qsize 3
2018-01-26 22:57:56,725 : INFO : PROGRESS: at 39.89% examples, 1336433 words/s, in_qsize 18, out_qsize 1
2018-01-26 22:57:57,734 : INFO : PROGRESS: at 40.79% examples, 1337006 words/s, in_qsize 17, out_qsize 2
2018-01-26 22:57:58,737 : INFO : PROGRESS: at 41.64% examples, 1336655 words/s, in_qsize 19, out_qsize 0
2018-01-26 22:57:59,755 : INFO : PROGRESS: at 42.35% examples, 1335389 words/s, in_qsize 18, out_qsize 1
2018-01-26 22:58:00,762 : INFO : PROGRESS: at 42.99% ex

2018-01-26 22:59:11,100 : INFO : worker thread finished; awaiting finish of 9 more threads
2018-01-26 22:59:11,113 : INFO : worker thread finished; awaiting finish of 8 more threads
2018-01-26 22:59:11,128 : INFO : worker thread finished; awaiting finish of 7 more threads
2018-01-26 22:59:11,134 : INFO : worker thread finished; awaiting finish of 6 more threads
2018-01-26 22:59:11,142 : INFO : worker thread finished; awaiting finish of 5 more threads
2018-01-26 22:59:11,143 : INFO : worker thread finished; awaiting finish of 4 more threads
2018-01-26 22:59:11,145 : INFO : worker thread finished; awaiting finish of 3 more threads
2018-01-26 22:59:11,147 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-01-26 22:59:11,162 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-01-26 22:59:11,170 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-01-26 22:59:11,171 : INFO : training on 207596775 raw words (151742453 effective word

2018-01-26 23:00:20,341 : INFO : PROGRESS: at 25.27% examples, 1125014 words/s, in_qsize 19, out_qsize 0
2018-01-26 23:00:21,345 : INFO : PROGRESS: at 25.70% examples, 1126757 words/s, in_qsize 19, out_qsize 0
2018-01-26 23:00:22,352 : INFO : PROGRESS: at 26.13% examples, 1128409 words/s, in_qsize 18, out_qsize 1
2018-01-26 23:00:23,354 : INFO : PROGRESS: at 26.56% examples, 1129534 words/s, in_qsize 19, out_qsize 0
2018-01-26 23:00:24,354 : INFO : PROGRESS: at 26.98% examples, 1131275 words/s, in_qsize 18, out_qsize 1
2018-01-26 23:00:25,357 : INFO : PROGRESS: at 27.43% examples, 1133695 words/s, in_qsize 18, out_qsize 1
2018-01-26 23:00:26,360 : INFO : PROGRESS: at 27.83% examples, 1135633 words/s, in_qsize 17, out_qsize 2
2018-01-26 23:00:27,366 : INFO : PROGRESS: at 28.21% examples, 1136048 words/s, in_qsize 17, out_qsize 2
2018-01-26 23:00:28,366 : INFO : PROGRESS: at 28.60% examples, 1136512 words/s, in_qsize 18, out_qsize 1
2018-01-26 23:00:29,368 : INFO : PROGRESS: at 29.00% ex

2018-01-26 23:01:39,899 : INFO : PROGRESS: at 57.39% examples, 1177163 words/s, in_qsize 19, out_qsize 0
2018-01-26 23:01:40,908 : INFO : PROGRESS: at 57.77% examples, 1177254 words/s, in_qsize 19, out_qsize 0
2018-01-26 23:01:41,922 : INFO : PROGRESS: at 58.11% examples, 1176260 words/s, in_qsize 17, out_qsize 2
2018-01-26 23:01:42,924 : INFO : PROGRESS: at 58.50% examples, 1176452 words/s, in_qsize 19, out_qsize 0
2018-01-26 23:01:43,926 : INFO : PROGRESS: at 58.91% examples, 1176143 words/s, in_qsize 19, out_qsize 0
2018-01-26 23:01:44,932 : INFO : PROGRESS: at 59.35% examples, 1176844 words/s, in_qsize 17, out_qsize 2
2018-01-26 23:01:45,934 : INFO : PROGRESS: at 59.79% examples, 1177426 words/s, in_qsize 19, out_qsize 0
2018-01-26 23:01:46,939 : INFO : PROGRESS: at 60.23% examples, 1178454 words/s, in_qsize 19, out_qsize 0
2018-01-26 23:01:47,939 : INFO : PROGRESS: at 60.66% examples, 1179160 words/s, in_qsize 20, out_qsize 0
2018-01-26 23:01:48,949 : INFO : PROGRESS: at 61.05% ex

2018-01-26 23:02:59,541 : INFO : PROGRESS: at 89.06% examples, 1187372 words/s, in_qsize 17, out_qsize 2
2018-01-26 23:03:00,549 : INFO : PROGRESS: at 89.52% examples, 1188008 words/s, in_qsize 19, out_qsize 0
2018-01-26 23:03:01,549 : INFO : PROGRESS: at 89.99% examples, 1188703 words/s, in_qsize 19, out_qsize 0
2018-01-26 23:03:02,550 : INFO : PROGRESS: at 90.44% examples, 1189519 words/s, in_qsize 19, out_qsize 0
2018-01-26 23:03:03,566 : INFO : PROGRESS: at 90.86% examples, 1189955 words/s, in_qsize 18, out_qsize 1
2018-01-26 23:03:04,568 : INFO : PROGRESS: at 91.09% examples, 1188430 words/s, in_qsize 19, out_qsize 0
2018-01-26 23:03:05,575 : INFO : PROGRESS: at 91.32% examples, 1186974 words/s, in_qsize 19, out_qsize 0
2018-01-26 23:03:06,586 : INFO : PROGRESS: at 91.64% examples, 1186622 words/s, in_qsize 18, out_qsize 1
2018-01-26 23:03:07,588 : INFO : PROGRESS: at 91.89% examples, 1185512 words/s, in_qsize 19, out_qsize 0
2018-01-26 23:03:08,594 : INFO : PROGRESS: at 92.21% ex

303489491

## Now, let's look at some output 
This first example shows a simple case of looking up words similar to the word `dirty`. All we need to do here is to call the `most_similar` function and provide the word `dirty` as the positive example. This returns the top 10 similar words. 

In [12]:

w1 = "dirty"
model.wv.most_similar (positive=w1)


2018-01-26 23:03:42,416 : INFO : precomputing L2-norms of word weight vectors


[('filthy', 0.871721625328064),
 ('stained', 0.7922376990318298),
 ('unclean', 0.7915753126144409),
 ('dusty', 0.7772612571716309),
 ('smelly', 0.7618112564086914),
 ('grubby', 0.7483716011047363),
 ('dingy', 0.7330487966537476),
 ('gross', 0.7239381074905396),
 ('grimy', 0.7228356599807739),
 ('disgusting', 0.7213647365570068)]

That looks pretty good, right? Let's look at a few more. Let's look at similarity for `polite`, `france` and `shocked`. 

In [50]:
# look up top 6 words similar to 'polite'
w1 = ["polite"]
model.wv.most_similar (positive=w1,topn=6)


[('courteous', 0.9174547791481018),
 ('friendly', 0.8309274911880493),
 ('cordial', 0.7990915179252625),
 ('professional', 0.7945970892906189),
 ('attentive', 0.7732747197151184),
 ('gracious', 0.7469891309738159)]

In [53]:
# look up top 6 words similar to 'france'
w1 = ["france"]
model.wv.most_similar (positive=w1,topn=6)


[('canada', 0.6603403091430664),
 ('germany', 0.6510637998580933),
 ('spain', 0.6431018114089966),
 ('barcelona', 0.61174076795578),
 ('mexico', 0.6070996522903442),
 ('rome', 0.6065913438796997)]

In [54]:
# look up top 6 words similar to 'shocked'
w1 = ["shocked"]
model.wv.most_similar (positive=w1,topn=6)


[('horrified', 0.80775386095047),
 ('amazed', 0.7797470092773438),
 ('astonished', 0.7748459577560425),
 ('dismayed', 0.7680633068084717),
 ('stunned', 0.7603034973144531),
 ('appalled', 0.7466776371002197)]

That's, nice. You can even specify several positive examples to get things that are related in the provided context and provide negative examples to say what should not be considered as related. In the example below we are asking for all items that *relate to bed* only:

In [55]:
# get everything related to stuff on the bed
w1 = ["bed",'sheet','pillow']
w2 = ['couch']
model.wv.most_similar (positive=w1,negative=w2,topn=10)


[('duvet', 0.7086508274078369),
 ('blanket', 0.7016597390174866),
 ('mattress', 0.7002605199813843),
 ('quilt', 0.6868821978569031),
 ('matress', 0.6777950525283813),
 ('pillowcase', 0.6413239240646362),
 ('sheets', 0.6382123827934265),
 ('foam', 0.6322235465049744),
 ('pillows', 0.6320573687553406),
 ('comforter', 0.5972476601600647)]

### Similarity between two words in the vocabulary

You can even use the Word2Vec model to return the similarity between two words that are present in the vocabulary. 

In [57]:
# similarity between two different words
model.wv.similarity(w1="dirty",w2="smelly")

0.76181122646029453

In [58]:
# similarity between two identical words
model.wv.similarity(w1="dirty",w2="dirty")

1.0000000000000002

In [59]:
# similarity between two unrelated words
model.wv.similarity(w1="dirty",w2="clean")

0.25355593501920781

Under the hood, the above three snippets computes the cosine similarity between the two specified words using word vectors of each. From the scores, it makes sense that `dirty` is highly similar to `smelly` but `dirty` is dissimilar to `clean`. If you do a similarity between two identical words, the score will be 1.0 as the range of the cosine similarity score will always be between [0.0-1.0]. You can read more about cosine similarity scoring [here](https://en.wikipedia.org/wiki/Cosine_similarity).

### Find the odd one out
You can even use Word2Vec to find odd items given a list of items.

In [63]:
# Which one is the odd one out in this list?
model.wv.doesnt_match(["cat","dog","france"])

'france'

In [77]:
# Which one is the odd one out in this list?
model.wv.doesnt_match(["bed","pillow","duvet","shower"])


'shower'

## Understanding some of the parameters
To train the model earlier, we had to set some parameters. Now, let's try to understand what some of them mean. For reference, this is the command that we used to train the model.

```
model = gensim.models.Word2Vec (documents, size=150, window=10, min_count=2, workers=10)
```

### `size`
The size of the dense vector to represent each token or word. If you have very limited data, then size should be a much smaller value. If you have lots of data, its good to experiment with various sizes. A value of 100-150 has worked well for me. 

### `window`
The maximum distance between the target word and its neighboring word. If your neighbor's position is greater than the maximum window width to the left and the right, then, some neighbors are not considered as being related to the target word. In theory, a smaller window should give you terms that are more related. If you have lots of data, then the window size should not matter too much, as long as its a decent sized window. 

### `min_count`
Minimium frequency count of words. The model would ignore words that do not statisfy the `min_count`. Extremely infrequent words are usually unimportant, so its best to get rid of those. Unless your dataset is really tiny, this does not really affect the model.

### `workers`
How many threads to use behind the scenes?


## When should you use Word2Vec?

There are many application scenarios for Word2Vec. Imagine if you need to build a sentiment lexicon. Training a Word2Vec model on large amounts of user reviews helps you achieve that. You have a lexicon for not just sentiment, but for most words in the vocabulary. 

Beyond, raw unstructured text data, you could also use Word2Vec for more structured data. For example, if you had tags for a million stackoverflow questions and answers, you could find tags that are related to a given tag and recommend the related ones for exploration. You can do this by treating each set of co-occuring tags as a "sentence" and train a Word2Vec model on this data. Granted, you still need a large number of examples to make it work. 
